In [39]:
import multiset

class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq

In [192]:
D = field('D', 1, 1, 0, 0)
F = field('F', 2, 2, 0, 0)
S = field('Pb_S_P', 3, 0, 0, 0)
V = field('Pb_V_P', 3, 1, 0, 0)
T = field('Pb_T_P', 3, 2, 0, 0)
Vp = field('Pb_Vp_P', 3, 1, 0, 0)
Sp = field('Pb_Sp_P', 3, 0, 0, 0)

In [211]:
# field_combo indicates how many of each field are contained in the term in question
field_dict = {'D': 2, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
print(field_dict.keys())
print(field_dict.values())

dict_keys(['D', 'F', 'Pb_S_P', 'Pb_V_P', 'Pb_T_P', 'Pb_Vp_P', 'Pb_Sp_P'])
dict_values([2, 1, 0, 0, 0, 0, 0])


In [212]:
list(field_dict.values())

[2, 1, 0, 0, 0, 0, 0]

In [217]:
def convert_symbolList_to_fieldList(symbol_list):
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 1, 0, 0)
    T = field('Pb_T_P', 3, 2, 0, 0)
    Vp = field('Pb_Vp_P', 3, 1, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    field_list = []
    for i in range(len(symbol_list)):
        if symbol_list[i] == 'D':
            field_list.append(D)
        if symbol_list[i] == 'F':
            field_list.append(F)
        if symbol_list[i] == 'Pb_S_P':
            field_list.append(S)
        if symbol_list[i] == 'Pb_V_P':
            field_list.append(V)
        if symbol_list[i] == 'Pb_T_P':
            field_list.append(T)
        if symbol_list[i] == 'Pb_Vp_P':
            field_list.append(Vp)
        if symbol_list[i] == 'Pb_Sp_P':
            field_list.append(Sp)
        
    return field_list


def lorentzRanks_list(field_dict):
    symbol_list = list(field_dict.keys())
    field_types = convert_symbolList_to_fieldList(symbol_list)
    numFields_list = list(field_dict.values())
    lor_ranks_list = [field_type.get_lorentz_rank() for field_type in field_types]
    N_fieldTypes = len(symbol_list)
    lorentzRanks = [num_fields*lor_rank for num_fields,lor_rank  in zip(numFields_list,lor_ranks_list)]
    # if field is 'F', 'T', number of lorentz indices is twice the number of fields
    #for i in range(N_fieldTypes):
    #    if symbol_list[i] == 'F' or symbol_list[i] == 'Pb_T_P':
    #        lorentzRanks[i] *= 2
    return lorentzRanks

     
def generate_lorentz_contractions(lorentzRanks_list):
    # EXPLANATION: recursively generateS all ways of lorentz contracting fields. generate a set of sets of contractions. all sets of
    # contractions should have the same number of contractions. take first index in lorentzRanks_list and contract it with every field with non-zero lorentz rank. 
    # for each such contraction, update the lorentzRanks_list by decrementing the lorentz rank of each contracted
    # field by one for each contracted index. append this contraction to each of the sets of contractions output 
    # by generate_lorentz_contractions() acting on the decremented lorentzRanks_list. 
    
    # base case: if the total number of free Lorentz indices is 0 or 1, return empty list of contraction sets 
    if sum(lorentzRanks_list) < 2:
        contraction_list_list = [[]]
        return contraction_list_list
    
    # set to store different sets of contractions
    contraction_list_list = []
    # variable to store number of distinct fields
    N_fieldTypes = len(lorentzRanks_list)
    
    # find first field of non-zero rank for first index of contraction
    for i in range(N_fieldTypes):
        if lorentzRanks_list[i] >= 1:
            i_start = i
            #print('i_start: ' + str(i_start))
            lorentzRanks_list[i_start] -= 1 #decrement lorentzRanks_list[i_start] by one
            break
            
    # find all ways of contracting first non-zero lorentz rank field with other fields or itself 
    for i in range(i_start, N_fieldTypes):
        #print(i)
        if lorentzRanks_list[i] >= 1:
            contraction = (i_start, i)
            lorentzRanks_list_old = lorentzRanks_list.copy()
            lorentzRanks_list_old[i] -= 1
            #print('lorentzRanks_list_old: ' + str(lorentzRanks_list_old))
            contraction_list_list_old = generate_lorentz_contractions(lorentzRanks_list_old)
            #print('contraction_list_list_old: ' + str(contraction_list_list_old))
            
            for contraction_list_old in contraction_list_list_old:
                #print(contraction_list_old)
                contraction_list = contraction_list_old
                contraction_list.append(contraction)
                #print(contraction_list)
                contraction_list_list.append(contraction_list) 
                #print(contraction_list_list)
    return contraction_list_list
        
        
def generate_lorentz_contractions_from_dict(field_dict):
    lorentzRanks = lorentzRanks_list(field_dict)
    contraction_list_list = generate_lorentz_contractions(lorentzRanks)
    return contraction_list_list
    
     
    

In [218]:
def reduced_set_of_contraction_sets(field_dict):
    contraction_list_list = generate_lorentz_contractions_from_dict(field_dict)
    contraction_set_set = set()
    for contraction_list in contraction_list_list:
    #print('contraction_list:' +str(contraction_list))
        contraction_set = FrozenMultiset(contraction_list)
        contraction_set_set.add(contraction_set)
    return contraction_set_set

In [219]:
convert_symbolList_to_fieldList(list(field_dict.keys()))


# Test 1

In [220]:
field_dict1 = {'D': 2, 'F': 1, 'Pb_S_P': 1, 'Pb_V_P': 2, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 1}
lorentzRanks = lorentzRanks_list(field_dict1)
lorentzRanks

[2, 2, 0, 2, 0, 0, 0]

In [221]:
generate_lorentz_contractions(lorentzRanks)

[[(3, 3), (1, 1), (0, 0)],
 [(1, 3), (1, 3), (0, 0)],
 [(3, 3), (0, 1), (0, 1)],
 [(1, 3), (0, 3), (0, 1)],
 [(1, 3), (0, 1), (0, 3)],
 [(1, 1), (0, 3), (0, 3)]]

In [222]:
field_dict2 = {'D': 2, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 2, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

generate_lorentz_contractions_from_dict(field_dict2)

[[(3, 3), (1, 1), (0, 0)],
 [(1, 3), (1, 3), (0, 0)],
 [(3, 3), (0, 1), (0, 1)],
 [(1, 3), (0, 3), (0, 1)],
 [(1, 3), (0, 1), (0, 3)],
 [(1, 1), (0, 3), (0, 3)]]

BUG: 4th and 5th contractions are redundant.

In [223]:
from multiset import FrozenMultiset
contraction_list_list = generate_lorentz_contractions_from_dict(field_dict2)
#print(contraction_list_list)
contraction_set_set = set()
for contraction_list in contraction_list_list:
    #print('contraction_list:' +str(contraction_list))
    contraction_set = FrozenMultiset(contraction_list)
    print('contraction_set:' +str(contraction_set))
    contraction_set_set.add(contraction_set)
    
contraction_set_set    
    

contraction_set:{(3, 3), (1, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 0)}
contraction_set:{(3, 3), (0, 1), (0, 1)}
contraction_set:{(1, 3), (0, 3), (0, 1)}
contraction_set:{(1, 3), (0, 1), (0, 3)}
contraction_set:{(1, 1), (0, 3), (0, 3)}


{FrozenMultiset({(1, 3): 2, (0, 0): 1}),
 FrozenMultiset({(1, 3): 1, (0, 3): 1, (0, 1): 1}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (0, 1): 2}),
 FrozenMultiset({(1, 1): 1, (0, 3): 2})}

Removed redundant contraction set, but is there a way to avoid producing it in the first place?

In [224]:
field_dict4 = {'D': 4, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 3, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

len(generate_lorentz_contractions_from_dict(field_dict4))

59

In [225]:
contraction_list_list4 = generate_lorentz_contractions_from_dict(field_dict4)
contraction_set_set4 = set()
for contraction_list in contraction_list_list4:
    #print('contraction_list:' +str(contraction_list))
    contraction_set = FrozenMultiset(contraction_list)
    print('contraction_set:' +str(contraction_set))
    contraction_set_set4.add(contraction_set)
    
#len(contraction_set_set4)
print(len(contraction_list_list4))
print(len(contraction_set_set4))

contraction_set:{(3, 3), (1, 1), (1, 1), (0, 0), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 0), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 0), (0, 0)}
contraction_set:{(1, 1), (1, 3), (1, 3), (0, 0), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 3), (0, 0), (0, 0)}
contraction_set:{(3, 3), (1, 1), (0, 1), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 1), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 1), (0, 3), (0, 1), (0, 0)}
contraction_set:{(1, 1), (1, 3), (0, 3), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 3), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 1), (0, 1), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 3), (0, 1), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 3), (0, 1), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 1), (0, 3), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 1), (0, 3), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 3), (0, 3), (0, 3), (0, 0)}
contraction_set:{(3, 3), (1, 1), (0, 1), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3)

# Test 2

In [226]:
field_dict3 = {'D': 0, 'F': 3, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

generate_lorentz_contractions_from_dict(field_dict3)

[[(1, 1), (1, 1), (1, 1)]]

BUG: can't distinguish between contractions on the same F and contractions on different F's.

# Test 3

In [228]:
field_dict5 = {'D': 3, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 3, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}

generate_lorentz_contractions_from_dict(field_dict5)

[[(3, 3), (1, 3), (1, 1), (0, 1), (0, 0)],
 [(3, 3), (1, 1), (1, 3), (0, 1), (0, 0)],
 [(1, 3), (1, 3), (1, 3), (0, 1), (0, 0)],
 [(3, 3), (1, 1), (1, 1), (0, 3), (0, 0)],
 [(1, 3), (1, 3), (1, 1), (0, 3), (0, 0)],
 [(1, 3), (1, 1), (1, 3), (0, 3), (0, 0)],
 [(1, 1), (1, 3), (1, 3), (0, 3), (0, 0)],
 [(3, 3), (1, 3), (1, 1), (0, 0), (0, 1)],
 [(3, 3), (1, 1), (1, 3), (0, 0), (0, 1)],
 [(1, 3), (1, 3), (1, 3), (0, 0), (0, 1)],
 [(3, 3), (1, 3), (0, 1), (0, 1), (0, 1)],
 [(3, 3), (1, 1), (0, 3), (0, 1), (0, 1)],
 [(1, 3), (1, 3), (0, 3), (0, 1), (0, 1)],
 [(3, 3), (1, 1), (0, 1), (0, 3), (0, 1)],
 [(1, 3), (1, 3), (0, 1), (0, 3), (0, 1)],
 [(1, 3), (1, 1), (0, 3), (0, 3), (0, 1)],
 [(1, 1), (1, 3), (0, 3), (0, 3), (0, 1)],
 [(3, 3), (1, 1), (1, 1), (0, 0), (0, 3)],
 [(1, 3), (1, 3), (1, 1), (0, 0), (0, 3)],
 [(1, 3), (1, 1), (1, 3), (0, 0), (0, 3)],
 [(1, 1), (1, 3), (1, 3), (0, 0), (0, 3)],
 [(3, 3), (1, 1), (0, 1), (0, 1), (0, 3)],
 [(1, 3), (1, 3), (0, 1), (0, 1), (0, 3)],
 [(1, 3), (

In [229]:
contraction_list_list5 = generate_lorentz_contractions_from_dict(field_dict5)
contraction_set_set5 = set()
for contraction_list in contraction_list_list5:
    #print('contraction_list:' +str(contraction_list))
    contraction_set = FrozenMultiset(contraction_list)
    print('contraction_set:' +str(contraction_set))
    contraction_set_set5.add(contraction_set)
    
#len(contraction_set_set4)
print(len(contraction_list_list5))
print(len(contraction_set_set5))
contraction_set_set5

contraction_set:{(3, 3), (1, 3), (1, 1), (0, 1), (0, 0)}
contraction_set:{(3, 3), (1, 1), (1, 3), (0, 1), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 3), (0, 1), (0, 0)}
contraction_set:{(3, 3), (1, 1), (1, 1), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 3), (0, 0)}
contraction_set:{(1, 3), (1, 3), (1, 1), (0, 3), (0, 0)}
contraction_set:{(1, 1), (1, 3), (1, 3), (0, 3), (0, 0)}
contraction_set:{(3, 3), (1, 3), (1, 1), (0, 0), (0, 1)}
contraction_set:{(3, 3), (1, 1), (1, 3), (0, 0), (0, 1)}
contraction_set:{(1, 3), (1, 3), (1, 3), (0, 0), (0, 1)}
contraction_set:{(3, 3), (1, 3), (0, 1), (0, 1), (0, 1)}
contraction_set:{(3, 3), (1, 1), (0, 3), (0, 1), (0, 1)}
contraction_set:{(1, 3), (1, 3), (0, 3), (0, 1), (0, 1)}
contraction_set:{(3, 3), (1, 1), (0, 1), (0, 1), (0, 3)}
contraction_set:{(1, 3), (1, 3), (0, 1), (0, 1), (0, 3)}
contraction_set:{(1, 3), (1, 1), (0, 3), (0, 3), (0, 1)}
contraction_set:{(1, 1), (1, 3), (0, 3), (0, 3), (0, 1)}
contraction_set:{(3, 3), (1, 1)

{FrozenMultiset({(1, 1): 2, (0, 3): 3}),
 FrozenMultiset({(3, 3): 1, (1, 1): 2, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (1, 1): 1, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (1, 1): 1, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(1, 3): 3, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (0, 1): 3}),
 FrozenMultiset({(1, 3): 1, (1, 1): 1, (0, 3): 2, (0, 1): 1})}

In [230]:
# test reduce_set_of_contraction_sets()
reduced_set_of_contraction_sets(field_dict5)

{FrozenMultiset({(1, 1): 2, (0, 3): 3}),
 FrozenMultiset({(3, 3): 1, (1, 1): 2, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (1, 1): 1, (0, 3): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (1, 1): 1, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(1, 3): 2, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(1, 3): 3, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 3): 1, (0, 1): 2}),
 FrozenMultiset({(3, 3): 1, (1, 3): 1, (0, 1): 3}),
 FrozenMultiset({(1, 3): 1, (1, 1): 1, (0, 3): 2, (0, 1): 1})}

# Generate all Field Combinations up to a Given Mass Dimension

In [231]:
def symbols_to_fields(field_combo_symbols):
    field_combo = []
    for field_symbol in field_combo_symbols:
        if field_symbol == 'D':
            field_combo.append(D)
            continue
        if field_symbol == 'F':
            field_combo.append(F)
            continue
        if field_symbol == 'Pb_S_P':
            field_combo.append(S)
            continue
        if field_symbol == 'Pb_V_P':
            field_combo.append(V)
            continue
        if field_symbol == 'Pb_T_P':
            field_combo.append(T)
            continue
        if field_symbol == 'Pb_Vp_P':
            field_combo.append(Vp)
            continue
        if field_symbol == 'Pb_Sp_P':
            field_combo.append(Sp)
            continue
         
    return field_combo

def totalMassDim(field_combo_symbols):
    field_combo = symbols_to_fields(field_combo_symbols)
    m = 0
    for field in field_combo:
        m += field.get_massDim()
    return m

def generate_field_combos(massDim):
    # EXPLANATION: Generate all multisets of the symbols [D, F, S, V, T, Vp, Sp] up to massDim. store in a set of 
    # multisets called set_of_field_multisets. generate new multisets by adding each field to multisets in a set
    # of multisets called front. remove a multiset from front after attempting to add each field to it. add an
    # element to front only if after a field has been added, its mass dimension is still less than massDim. returns
    # set_of_multisets, a dictionary mapping each mass dimension up to massDim to a set of multisets. 
    
    # declare field types
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 0, 0, 0)
    T = field('Pb_T_P', 3, 0, 0, 0)
    Vp = field('Pb_Vp_P', 3, 0, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    # make list of field types
    fields = [D, F, S, V, T, Vp, Sp] 
    
    # set of multisets
    set_of_field_multisets = {}
    for m in range(massDim + 1): #set of multisets of field strings, arranged by massDim
        set_of_field_multisets[m] = set()
    front = set() #set of multisets of field strings, set of multisets to add to to make new multisets
    
    # initialize front
    for item in fields: 
        m = item.get_massDim()
        if m <= massDim:
            front.add(FrozenMultiset([item.get_symbol()]))
            set_of_field_multisets[m].add(FrozenMultiset([item.get_symbol()]))

    # build set_of_multisets
    while front:
        #print('')
        #print('front: ' + str(front))
        front_new = front.copy()
        for field_multiset in front:
            #print('field_multiset: ' + str(field_multiset))
            for item in fields:
                #print('field item: ' + str(item.get_symbol()))
                field_multiset_new = field_multiset.combine(FrozenMultiset([item.get_symbol()]))
                #print('field_multiset_new: ' + str(field_multiset_new))
                m = totalMassDim(field_multiset_new)
                if m < massDim:
                    set_of_field_multisets[m].add(field_multiset_new)
                    front_new.add(field_multiset_new)
                    #print('ADD TO FRONT: ' + str(field_multiset_new))
                if m == massDim:
                    set_of_field_multisets[m].add(field_multiset_new)
            front_new.remove(field_multiset)
        front = front_new
        #print('front_new: ' + str(front_new))
        #print('')
    
    return set_of_field_multisets
                    
                    
    
    
    
    
    

In [232]:
massDim = 5
for key in generate_field_combos(massDim):
    print(str(key) + ': ' + str(generate_field_combos(massDim)[key]))
    print('')

0: set()

1: {FrozenMultiset({'D': 1})}

2: {FrozenMultiset({'F': 1}), FrozenMultiset({'D': 2})}

3: {FrozenMultiset({'Pb_Vp_P': 1}), FrozenMultiset({'Pb_T_P': 1}), FrozenMultiset({'D': 1, 'F': 1}), FrozenMultiset({'D': 3}), FrozenMultiset({'Pb_Sp_P': 1}), FrozenMultiset({'Pb_S_P': 1}), FrozenMultiset({'Pb_V_P': 1})}

4: {FrozenMultiset({'Pb_T_P': 1, 'D': 1}), FrozenMultiset({'D': 1, 'Pb_Sp_P': 1}), FrozenMultiset({'Pb_Vp_P': 1, 'D': 1}), FrozenMultiset({'F': 2}), FrozenMultiset({'D': 2, 'F': 1}), FrozenMultiset({'D': 1, 'Pb_S_P': 1}), FrozenMultiset({'D': 1, 'Pb_V_P': 1}), FrozenMultiset({'D': 4})}

5: {FrozenMultiset({'D': 2, 'Pb_S_P': 1}), FrozenMultiset({'D': 1, 'F': 2}), FrozenMultiset({'F': 1, 'Pb_Sp_P': 1}), FrozenMultiset({'Pb_Vp_P': 1, 'D': 2}), FrozenMultiset({'F': 1, 'Pb_S_P': 1}), FrozenMultiset({'F': 1, 'Pb_V_P': 1}), FrozenMultiset({'Pb_Vp_P': 1, 'F': 1}), FrozenMultiset({'D': 2, 'Pb_Sp_P': 1}), FrozenMultiset({'D': 5}), FrozenMultiset({'Pb_T_P': 1, 'D': 2}), FrozenMultis

Tested for massdim up to 5

# Generate All Lorentz Contractions for Every Field Combination up to a Given Mass Dimension

In [233]:
def generate_lorentz_contractions_from_massDim(massDim, up_to = True):
    # ARGUMENTS
    # massDim: mass dimension 
    # up_to = whether to produce lorentz contracted field combos up to and including massDim (up_to = True) or only
    # at massDim (up_to = False)
    #
    # EXPLANATION: generates all lorentz contractions of all field combinations at, or up to and including, massDim.
    # if up_to = True, the function outputs a dictionary with keys m corresponding to every mass dimension up to and
    # including massDim; the values are themselves dictionaries, with keys equal to different field combinations
    # of mass dimension m, and values equal to sets of multisets of lorentz contractions of fields in the combo. 
    # each multiset corresponds to a distinct way of contracting the fields in a given field combination. if 
    # up_to = False, then simply return the dictionary corresponding to mass dimension massDim, whose keys are
    # field combinations of mass dimension massDim and whose values are sets of multisets of lorentz contractions. 
    
    if up_to == True:
        lorentz_contracted_combos = {}
        set_of_field_multisets = generate_field_combos(massDim)
        for m in set_of_field_multisets.keys():
            lorentz_contracted_combos_m = {}
            for combo in set_of_field_multisets[m]:
                field_combo = dict(combo.items())
                lorentz_contracted_combos_m[combo] = reduced_set_of_contraction_sets(field_combo)
            lorentz_contracted_combos[m] = lorentz_contracted_combos_m
        return lorentz_contracted_combos
    else: 
        lorentz_contracted_combos_m = {}
        set_of_field_multisets = generate_field_combos(massDim)
        for combo in set_of_field_multisets[massDim]:
            field_combo = dict(combo.items())
            lorentz_contracted_combos_m[combo] = reduced_set_of_contraction_sets(field_combo)
        return lorentz_contracted_combos_m

In [234]:
massDim = 6
lorentz_contraction_dict = generate_lorentz_contractions_from_massDim(massDim, up_to = False)
for field_combo in lorentz_contraction_dict.keys():
    print(field_combo)
    print(lorentz_contraction_dict[field_combo])
    
    

{F, Pb_S_P, D}
{FrozenMultiset({(0, 0): 1}), FrozenMultiset({(0, 2): 1})}
{Pb_Sp_P, Pb_Sp_P}
{FrozenMultiset({})}
{Pb_T_P, Pb_V_P}
{FrozenMultiset({(0, 1): 1}), FrozenMultiset({(0, 0): 1})}
{F, F, F}
{FrozenMultiset({(0, 0): 3})}
{Pb_Vp_P, Pb_Vp_P}
{FrozenMultiset({(0, 0): 1})}
{Pb_S_P, Pb_V_P}
{FrozenMultiset({})}
{D, Pb_Sp_P, F}
{FrozenMultiset({(0, 2): 1})}
{Pb_Vp_P, Pb_S_P}
{FrozenMultiset({})}
{Pb_V_P, Pb_V_P}
{FrozenMultiset({(0, 0): 1})}
{D, D, D, D, D, D}
{FrozenMultiset({(0, 0): 3})}
{D, D, D, Pb_Vp_P}
{FrozenMultiset({(0, 1): 1, (0, 0): 1})}
{Pb_S_P, Pb_S_P}
{FrozenMultiset({})}
{Pb_Sp_P, Pb_S_P}
{FrozenMultiset({})}
{Pb_Vp_P, Pb_Sp_P}
{FrozenMultiset({})}
{Pb_T_P, F, D}
{FrozenMultiset({(1, 2): 1, (0, 0): 1}), FrozenMultiset({(0, 1): 2}), FrozenMultiset({(1, 1): 1, (0, 0): 1}), FrozenMultiset({(0, 2): 1, (0, 1): 1})}
{Pb_Vp_P, Pb_V_P}
{FrozenMultiset({(0, 1): 1})}
{Pb_Vp_P, D, F}
{FrozenMultiset({(1, 2): 1, (0, 2): 1}), FrozenMultiset({(2, 2): 1, (0, 1): 1})}
{Pb_Sp_P, Pb_V_